# Les Finesses de Python

## Microclub 1er juin 2018

## Philippe Guglielmetti

---

## Pourquoi encore Python ?

> Il ne vaut pas la peine de connaître un langage qui ne modifie pas votre façon de penser la programmation.
> ([Alan Perlis](https://www.drgoulu.com/2008/01/21/perlisismes-les-dictons-informatiques-dalan-perlis/))

* #4 au [TIOBE](https://www.tiobe.com/tiobe-index/) (nombre de développeurs)
* #2 au [Madnight] (https://madnight.github.io/githut/#/issues/2018/1) (activité sur GitHub)
* #1 au [PYPL](http://pypl.github.io/PYPL.html) PopularitY of Programming Language (nombre de tutoriels suivis)



>Python est un langage de programmation objet, multi-paradigme ([wikipedia](https://fr.wikipedia.org/wiki/Python_(langage)))

= on peut programmer :
1. [comme une patate](http://entrian.com/goto/)
2. comme on en a l’habitude (classes, objets, ...)
3. comme on en a pas l’habitude
4. comme ~~un dieu~~ [Guido van Rossum](https://fr.wikipedia.org/wiki/Guido_van_Rossum)

---

## Classes et objets

In [1]:
256+1 is 257

False

In [2]:
255+1 is 256

True

In [3]:
type(256)

int

In [4]:
type(int)

type

In [5]:
type(type(int))

type

In [6]:
def f(n):
    """factorielle récursive"""
    return 1 if n<2 else n*f(n-1)
type(f)

function

In [7]:
type(type(f))

type

In [8]:
type(type(int)) is type(type(f))

True

## Introspection

In [9]:
print(dir(256)) # membres de l'objet

['__abs__', '__add__', '__and__', '__bool__', '__ceil__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__float__', '__floor__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getnewargs__', '__gt__', '__hash__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__le__', '__lshift__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rlshift__', '__rmod__', '__rmul__', '__ror__', '__round__', '__rpow__', '__rrshift__', '__rshift__', '__rsub__', '__rtruediv__', '__rxor__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__trunc__', '__xor__', 'bit_length', 'conjugate', 'denominator', 'from_bytes', 'imag', 'numerator', 'real', 'to_bytes']


In [10]:
print(dir(f)) # membres de l'objet

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [11]:
f.__doc__

'factorielle récursive'

In [12]:
import inspect # https://docs.python.org/3/library/inspect.html
inspect.getsourcelines(f)

(['def f(n):\n',
  '    """factorielle récursive"""\n',
  '    return 1 if n<2 else n*f(n-1)\n'],
 1)

## programmation fonctionnelle

In [38]:
for i in range(10):
    print(f(i), end=" ")

1 1 2 6 24 120 720 5040 40320 362880 

In [41]:
r=range(10)
print(r)
print(_ for _ in r)
print([_ for _ in r])

range(0, 10)
<generator object <genexpr> at 0x00000203A9AC63B8>
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [40]:
res=list(map(f,r))
res

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

In [14]:
import operator
from functools import reduce # était prédéfini en python 2.7
reduce(operator.add,res)


409114

## [Les patrons de conception](https://fr.wikipedia.org/wiki/Patron_de_conception)

"Gang of four" (GOF) Erich Gamma, Richard Helm, Ralph Johnson, John Vlissides 
“Design Patterns - Catalogue de modèles de conceptions réutilisables” 
Vuibert, 1999, 490 p 

### <a href="https://fr.wikipedia.org/wiki/D%C3%A9corateur_(patron_de_conception)">Décorateur</a>

Le pattern Décorateur est l'un des vingt-trois patterns GOF. Il résout les problématiques suivantes :

* L'ajout (et la suppression) des responsabilités à un objet dynamiquement au moment de l'exécution.
* Il constitue une alternative aux sous-classes pour une surcharge flexible des fonctionnalités.

Quand on utilise l'héritage, les différentes sous-classes étendent une classe mère en différentes manières. Mais une extension est attachée à la classe au moment de la compilation, et ne peut pas changer à l'exécution.

In [15]:
class Animal(object):
    def __init__(self, name):
        self.name=name

chien=Animal("chien")
chien.fait=lambda: "ouaf ouaf"

canard=Animal("canard")
canard.fait=lambda: "coin coin"

animaux=[chien, canard]
for a in animaux:
    print(" ".join(["le",a.name,"fait",a.fait()]))

le chien fait ouaf ouaf
le canard fait coin coin


## [Librairies](https://docs.python.org/3/library/)
quelques librairies standart fort utiles:

### [array](https://docs.python.org/3/library/array.html)
tableaux (comme des listes, mais type fixe)

In [8]:
import sys
from array import array
l=list(range(1000))
print(sys.getsizeof(l),sys.getsizeof(array('i',l)))

9112 4064


### [re](https://docs.python.org/3/library/re.html)
regular expressions